# [STARTER] Exercise - Building a Multi-Step State Machine Agent

In this exercise, you will build an agent that manages a multi-step workflow using a state machine. You’ll define a custom state schema, implement step logic, connect steps (including conditional routing and loops), and run the workflow to process user input through several transformations.

## Challenge

You have learned how to use a state machine to manage workflow steps and transitions. Now, your challenge is to:

- Define a state schema with multiple fields (e.g., user_query, instructions, messages, current_tool_calls).
- Implement at least three step functions:
    - Prepare Messages: Assemble the conversation history and any required context for the LLM.
    - LLM: Call the language model to generate a response or tool call.
    - Tools: Execute any required tool calls and update the state with results.
- Connect steps to form a workflow, including:
    - Entrypoint and Termination steps to start and end the workflow.
    - Conditional routing: If the LLM response includes tool calls, route to the Tools step; otherwise, proceed to Termination.
    - Looping: After executing tools, return to the LLM step to continue the workflow until there are no more tool calls.
- Run your state machine with a sample input and inspect the state transitions and snapshots to understand how your agent processes a task step by step.


## Define a State Schema

Create a TypedDict to represent the agent’s state, including fields for the user query, instructions, message history, and any pending tool calls.

## Setup
First, let's import the necessary libraries:

In [14]:
from typing import TypedDict, List, Optional, Union
import json
from dotenv import load_dotenv
import os

from lib.state_machine import StateMachine, Step, EntryPoint, Termination, Run
from lib.llm import LLM
from lib.messages import AIMessage, UserMessage, SystemMessage, ToolMessage
from lib.tooling import Tool, ToolCall, tool

In [15]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_api_key

'voc-2015200036168865441126268e0ed5be47456.24871461'

In [16]:
# TODO: Define the AgentState TypedDict
# Include fields for user_query, instructions, messages, and current_tool_calls

class AgentState(TypedDict):
    user_query: str
    instructions: str
    messages: List[str]
    pending_tool_calls: Optional[List[ToolCall]]    

## Define the Tools you will use

Feel free to modify to add any tool you want

In [17]:
@tool
def get_games(num_games:int=1, top:bool=True) -> str:
    """
    Returns the top or bottom N games with highest or lowest scores.    
    args:
        num_games (int): Number of games to return (default is 1)
        top (bool): If True, return top games, otherwise return bottom (default is True)
    """
    data = [
        {"Game": "The Legend of Zelda: Breath of the Wild", "Platform": "Switch", "Score": 98},
        {"Game": "Super Mario Odyssey", "Platform": "Switch", "Score": 97},
        {"Game": "Metroid Prime", "Platform": "GameCube", "Score": 97},
        {"Game": "Super Smash Bros. Brawl", "Platform": "Wii", "Score": 93},
        {"Game": "Mario Kart 8 Deluxe", "Platform": "Switch", "Score": 92},
        {"Game": "Fire Emblem: Awakening", "Platform": "3DS", "Score": 92},
        {"Game": "Donkey Kong Country Returns", "Platform": "Wii", "Score": 87},
        {"Game": "Luigi's Mansion 3", "Platform": "Switch", "Score": 86},
        {"Game": "Pikmin 3", "Platform": "Wii U", "Score": 85},
        {"Game": "Animal Crossing: New Leaf", "Platform": "3DS", "Score": 88}
    ]
    # Sort the games list by Score
    # If top is True, descending order
    sorted_games = sorted(data, key=lambda x: x['Score'], reverse=top)
    
    # Return the N games
    return sorted_games[:num_games]

In [18]:
# TODO: Add as many tools as you want
# Use the @tool decorator and implement functions like get_games

tools = [get_games]

## Create the Steps

Write functions for each step in your workflow:


**Prepare Messages**: Build the message list for the LLM.

In [19]:
# TODO: Create the prepare_messages_step function
# This function should take AgentState and return AgentState with prepared messages
# Use instructions to create the SystemMessage and user_query to create UserMessage
# Then return AgentState with the messages list with the SystemMessage and UserMessage

def prepare_messages_step(state: AgentState) -> AgentState:
    user_message = UserMessage(content=state["user_query"])
    system_message = SystemMessage(
        content=f'You are a game assistant expert that follows these instructions: {state["instructions"]}'
    )
    
    return {
        "messages": [user_message, system_message],
    }

**LLM Step**: Call the language model and check for tool calls.

In [20]:
# TODO: Create the llm_step function
# This function should process the state through an LLM and handle tool calls
# It should append the AIMessage to the messages list 
# and return the State with the messages and the current_tool_calls.
# You can get the tool_calls object accessing it from the llm invoke response: `response.tool_calls`

def llm_step(state: AgentState) -> AgentState:
    print('llm_step 1')
    llm = LLM(api_key=openai_api_key, tools=tools)
    print('llm_step 2', state['messages'])
    ai_message = llm.invoke(state["messages"])
    print('llm_step 3')

    return {
        "user_query": state["user_query"],
        "instructions": state["instructions"],
        "messages": state["messages"].append(ai_message),
        "pending_tool_calls": ai_message.tool_calls
    }
    

**Tool Step**: Execute any tool calls and update the state.

In [21]:
# TODO: Create the tool_step function
# This function should execute any pending tool calls and update the state
# Make sure to iterate over tool_calls object
# Extend the messages list from the state with all ToolMessages
# Don't forget to set current_tool_calls to None

def tool_step(state: AgentState) -> AgentState:
    tool_messages = []
    for tool_call in state["pending_tool_calls"]:
        tool_call_name = tool_call.function.name
        tool_call_args = json.load(tool_call.function.arguments)
        tool_call_id = tool_call.id
        for tool in tools:
            if tool.name == tool_call_name:
                result = tool(**tool_call_args)
                tool_message = ToolMessage(content=result, name=tool_call_name, tool_call_id=tool_call_id)
                tool_messages.append(tool_message)

    return {
        "user_query": state["user_query"],
        "instructions": state["instructions"],
        "messages": state["messages"] + tool_messages,
        "pending_tool_calls": []
    }



## Build and Connect the State Machine

Add your steps to the state machine, and connect them with transitions. Use conditional routing to decide whether to call tools or terminate, and loop as needed.

In [22]:
# TODO: Initialize the StateMachine with the AgentState
workflow = StateMachine[AgentState](AgentState)

In [23]:
entry = EntryPoint[AgentState]()
message_prep = Step[AgentState]("message_prep", prepare_messages_step)
llm_processor = Step[AgentState]("llm_processor", llm_step)
tool_executor = Step[AgentState]("tool_executor", tool_step)
termination = Termination[AgentState]()

# TODO: Add all the steps to the workflow using workflow.add_steps   
workflow.add_steps([entry, message_prep, llm_processor, tool_executor, termination])

In [24]:
# Add transitions
workflow.connect(entry, message_prep)
workflow.connect(message_prep, llm_processor)

# Transition based on whether there are tool calls
def check_tool_calls(state: AgentState) -> Union[Step[AgentState], str]:
    """Transition logic: Check if there are tool calls"""
    if state.get("current_tool_calls"):
        return tool_executor
    return termination

# Routing: If tool calls -> tool_executor
workflow.connect(
    source=llm_processor, 
    targets=[tool_executor, termination], 
    condition=check_tool_calls
)

# Looping: Go back to llm after tool execution
workflow.connect(
    source=tool_executor, 
    targets=llm_processor
)

## Run the Workflow

In [25]:
initial_state: AgentState = {
    "user_query": "What's the best game in the dataset?",
    "instructions": "You can bring insights about a game dataset based on users questions",
    "messages": [],
}

In [26]:
run_object = workflow.run(initial_state)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
llm_step 1
llm_step 2 [UserMessage(role='user', content="What's the best game in the dataset?"), SystemMessage(role='system', content='You are a game assistant expert that follows these instructions: You can bring insights about a game dataset based on users questions')]
invoke
else in invoke


APIConnectionError: Connection error.

In [ ]:
run_object.get_final_state()["messages"]

In [ ]:
# TODO: Create more test cases
# Initialize the state and run the workflow with another sample query

## Optional 

Create an Agent class to encapsulate State Machine logic. Then try adding more tools, and experiment with different user queries to see how the workflow adapts.

In [ ]:
# TODO: Implement your Agent
# Develope the following methods: _prepare_messages_step, _llm_step, _tool_step, _create_state_machine
class Agent:
    def __init__(self, 
                 model_name: str,
                 instructions: str, 
                 tools: List[Tool] = None,
                 temperature: float = 0.7):
        """
        Initialize an Agent instance
        
        Args:
            model_name: Name/identifier of the LLM model to use
            instructions: System instructions for the agent
            tools: Optional list of tools available to the agent
            temperature: Temperature parameter for LLM (default: 0.7)
        """
        self.instructions = instructions
        self.tools = tools if tools else []
        self.model_name = model_name
        self.temperature = temperature
                
        # Initialize state machine
        self.workflow = self._create_state_machine()

    def _prepare_messages_step(self, state: AgentState) -> AgentState:
        # TODO
        pass

    def _llm_step(self, state: AgentState) -> AgentState:
        # TODO
        pass

    def _tool_step(self, state: AgentState) -> AgentState:
        # TODO
        pass

    def _create_state_machine(self) -> StateMachine[AgentState]:
        # TODO
        pass

    def invoke(self, query: str) -> Run:
        """
        Run the agent on a query
        
        Args:
            query: The user's query to process
            
        Returns:
            The final run object after processing
        """

        initial_state: AgentState = {
            "user_query": query,
            "instructions": self.instructions,
            "messages": [],
        }

        run_object = self.workflow.run(initial_state)

        return run_object
